In [1]:
# learning curve, fitting,feature importance. rmse,mse, etc..
#import pmdarima as pm
import matplotlib.pyplot as plt
import pandas as pd
# https://www.kaggle.com/code/rtatman/lightgbm-hyperparameter-optimisation-lb-0-761/notebook
import xgboost as xgb
import lightgbm as lgb
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)   
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

C:\Users\ahmed.ewis\Anaconda3_ts_3.8\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\ahmed.ewis\Anaconda3_ts_3.8\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\ahmed.ewis\Anaconda3_ts_3.8\lib\site-packages\dask\dataframe\utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
df = pd.read_excel('Interpolation_Average_2013_2015.xlsx',sheet_name='Sheet2')
df

,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,CO2,PM10,SO2,H2S,NO,NOX,NO2,O3,CO,THC
0,2013-01-27 00:00:00,114,1.7,18.1,5,100,380,101.000000,0.001000,0.003,0.005,0.023,0.017000,2.600000e-02,0.58,2.170000
1,2013-01-27 01:00:00,151,0.4,17.8,3,89,410,101.000000,0.003000,0.017,0.065,0.151,0.086000,1.000000e-03,1.74,2.830000
2,2013-01-27 02:00:00,175,0.3,17.8,2,88,392,100.000000,0.002000,0.004,0.019,0.072,0.053000,2.000000e-03,0.95,3.770000
3,2013-01-27 03:00:00,264,0.8,16.8,2,92,403,92.000000,0.002000,0.014,0.051,0.123,0.072000,1.000000e-03,1.43,2.720000
4,2013-01-27 04:00:00,187,0.4,16.2,2,94,400,69.000000,0.002000,0.018,0.041,0.105,0.063000,1.000000e-03,1.32,2.520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22774,2015-12-31 18:00:00,85,1.4,14.6,10,76,404,344.238675,0.006386,0.008,0.014,0.081,0.076199,1.747050e-03,2.73,1.464756
22775,2015-12-31 19:00:00,99,1.3,14.6,9,75,410,281.815817,0.006301,0.009,0.023,0.100,0.098153,1.108100e-04,2.81,1.903803
22776,2015-12-31 20:00:00,210,0.7,14.5,10,79,424,51.079508,0.006520,0.009,0.064,0.164,0.106758,8.930000e-06,2.96,2.039618
22777,2015-12-31 21:00:00,185,0.8,14.6,10,81,428,120.974071,0.006437,0.009,0.077,0.182,0.087641,1.000000e-08,3.25,1.754740


In [3]:
df.columns

Index(['MeasurementDateTime', 'WD-Hour', 'WS-Hour', 'Temp-Hour', 'SR-Hour', 'RH-Hour', 'CO2', 'PM10', 'SO2', 'H2S', 'NO', 'NOX', 'NO2', 'O3', 'CO', 'THC'], dtype='object')

In [4]:
cols = ['MeasurementDateTime','WD-Hour', 'WS-Hour', 'Temp-Hour', 'SR-Hour', 'RH-Hour','NO2', 'O3'
]

In [5]:
df = df[cols]
df

,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3
0,2013-01-27 00:00:00,114,1.7,18.1,5,100,0.017000,2.600000e-02
1,2013-01-27 01:00:00,151,0.4,17.8,3,89,0.086000,1.000000e-03
2,2013-01-27 02:00:00,175,0.3,17.8,2,88,0.053000,2.000000e-03
3,2013-01-27 03:00:00,264,0.8,16.8,2,92,0.072000,1.000000e-03
4,2013-01-27 04:00:00,187,0.4,16.2,2,94,0.063000,1.000000e-03
...,...,...,...,...,...,...,...,...
22774,2015-12-31 18:00:00,85,1.4,14.6,10,76,0.076199,1.747050e-03
22775,2015-12-31 19:00:00,99,1.3,14.6,9,75,0.098153,1.108100e-04
22776,2015-12-31 20:00:00,210,0.7,14.5,10,79,0.106758,8.930000e-06
22777,2015-12-31 21:00:00,185,0.8,14.6,10,81,0.087641,1.000000e-08


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22779 entries, 0 to 22778
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   MeasurementDateTime  22779 non-null  datetime64[ns]
 1   WD-Hour              22779 non-null  int64         
 2   WS-Hour              22779 non-null  float64       
 3   Temp-Hour            22779 non-null  float64       
 4   SR-Hour              22779 non-null  int64         
 5   RH-Hour              22779 non-null  int64         
 6   NO2                  22779 non-null  float64       
 7   O3                   22779 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 1.4 MB


In [7]:
df['MeasurementDateTime'] = pd.to_datetime(df['MeasurementDateTime'], errors='coerce')
df

,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3
0,2013-01-27 00:00:00,114,1.7,18.1,5,100,0.017000,2.600000e-02
1,2013-01-27 01:00:00,151,0.4,17.8,3,89,0.086000,1.000000e-03
2,2013-01-27 02:00:00,175,0.3,17.8,2,88,0.053000,2.000000e-03
3,2013-01-27 03:00:00,264,0.8,16.8,2,92,0.072000,1.000000e-03
4,2013-01-27 04:00:00,187,0.4,16.2,2,94,0.063000,1.000000e-03
...,...,...,...,...,...,...,...,...
22774,2015-12-31 18:00:00,85,1.4,14.6,10,76,0.076199,1.747050e-03
22775,2015-12-31 19:00:00,99,1.3,14.6,9,75,0.098153,1.108100e-04
22776,2015-12-31 20:00:00,210,0.7,14.5,10,79,0.106758,8.930000e-06
22777,2015-12-31 21:00:00,185,0.8,14.6,10,81,0.087641,1.000000e-08


In [8]:
df['date'] = df['MeasurementDateTime']
df

,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3,date
0,2013-01-27 00:00:00,114,1.7,18.1,5,100,0.017000,2.600000e-02,2013-01-27 00:00:00
1,2013-01-27 01:00:00,151,0.4,17.8,3,89,0.086000,1.000000e-03,2013-01-27 01:00:00
2,2013-01-27 02:00:00,175,0.3,17.8,2,88,0.053000,2.000000e-03,2013-01-27 02:00:00
3,2013-01-27 03:00:00,264,0.8,16.8,2,92,0.072000,1.000000e-03,2013-01-27 03:00:00
4,2013-01-27 04:00:00,187,0.4,16.2,2,94,0.063000,1.000000e-03,2013-01-27 04:00:00
...,...,...,...,...,...,...,...,...,...
22774,2015-12-31 18:00:00,85,1.4,14.6,10,76,0.076199,1.747050e-03,2015-12-31 18:00:00
22775,2015-12-31 19:00:00,99,1.3,14.6,9,75,0.098153,1.108100e-04,2015-12-31 19:00:00
22776,2015-12-31 20:00:00,210,0.7,14.5,10,79,0.106758,8.930000e-06,2015-12-31 20:00:00
22777,2015-12-31 21:00:00,185,0.8,14.6,10,81,0.087641,1.000000e-08,2015-12-31 21:00:00


In [9]:
from datetime import datetime
#df['MeasurementDateTime'] = df.index
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['dayofyear'] = df['date'].dt.dayofyear
df['dayofmonth'] = df['date'].dt.day
df['datehour'] = df['date'].dt.hour
df['weekofyear'] = df['date'].dt.weekofyear
df

,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3,date,dayofweek,quarter,month,year,dayofyear,dayofmonth,datehour,weekofyear
0,2013-01-27 00:00:00,114,1.7,18.1,5,100,0.017000,2.600000e-02,2013-01-27 00:00:00,6,1,1,2013,27,27,0,4
1,2013-01-27 01:00:00,151,0.4,17.8,3,89,0.086000,1.000000e-03,2013-01-27 01:00:00,6,1,1,2013,27,27,1,4
2,2013-01-27 02:00:00,175,0.3,17.8,2,88,0.053000,2.000000e-03,2013-01-27 02:00:00,6,1,1,2013,27,27,2,4
3,2013-01-27 03:00:00,264,0.8,16.8,2,92,0.072000,1.000000e-03,2013-01-27 03:00:00,6,1,1,2013,27,27,3,4
4,2013-01-27 04:00:00,187,0.4,16.2,2,94,0.063000,1.000000e-03,2013-01-27 04:00:00,6,1,1,2013,27,27,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22774,2015-12-31 18:00:00,85,1.4,14.6,10,76,0.076199,1.747050e-03,2015-12-31 18:00:00,3,4,12,2015,365,31,18,53
22775,2015-12-31 19:00:00,99,1.3,14.6,9,75,0.098153,1.108100e-04,2015-12-31 19:00:00,3,4,12,2015,365,31,19,53
22776,2015-12-31 20:00:00,210,0.7,14.5,10,79,0.106758,8.930000e-06,2015-12-31 20:00:00,3,4,12,2015,365,31,20,53
22777,2015-12-31 21:00:00,185,0.8,14.6,10,81,0.087641,1.000000e-08,2015-12-31 21:00:00,3,4,12,2015,365,31,21,53


In [10]:
df.columns

Index(['MeasurementDateTime', 'WD-Hour', 'WS-Hour', 'Temp-Hour', 'SR-Hour', 'RH-Hour', 'NO2', 'O3', 'date', 'dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'datehour', 'weekofyear'], dtype='object')

In [11]:
df = df.set_index('date').asfreq('h')
df


,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3,dayofweek,quarter,month,year,dayofyear,dayofmonth,datehour,weekofyear
date,,,,,,,,,,,,,,,,
2013-01-27 00:00:00,2013-01-27 00:00:00,114.0,1.7,18.1,5.0,100.0,0.017000,2.600000e-02,6.0,1.0,1.0,2013.0,27.0,27.0,0.0,4.0
2013-01-27 01:00:00,2013-01-27 01:00:00,151.0,0.4,17.8,3.0,89.0,0.086000,1.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,1.0,4.0
2013-01-27 02:00:00,2013-01-27 02:00:00,175.0,0.3,17.8,2.0,88.0,0.053000,2.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,2.0,4.0
2013-01-27 03:00:00,2013-01-27 03:00:00,264.0,0.8,16.8,2.0,92.0,0.072000,1.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,3.0,4.0
2013-01-27 04:00:00,2013-01-27 04:00:00,187.0,0.4,16.2,2.0,94.0,0.063000,1.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,2015-12-31 18:00:00,85.0,1.4,14.6,10.0,76.0,0.076199,1.747050e-03,3.0,4.0,12.0,2015.0,365.0,31.0,18.0,53.0
2015-12-31 19:00:00,2015-12-31 19:00:00,99.0,1.3,14.6,9.0,75.0,0.098153,1.108100e-04,3.0,4.0,12.0,2015.0,365.0,31.0,19.0,53.0
2015-12-31 20:00:00,2015-12-31 20:00:00,210.0,0.7,14.5,10.0,79.0,0.106758,8.930000e-06,3.0,4.0,12.0,2015.0,365.0,31.0,20.0,53.0


In [12]:
df.isna().sum()

MeasurementDateTime    2876
WD-Hour                2876
WS-Hour                2876
Temp-Hour              2876
SR-Hour                2876
RH-Hour                2876
NO2                    2876
O3                     2876
dayofweek              2876
quarter                2876
month                  2876
year                   2876
dayofyear              2876
dayofmonth             2876
datehour               2876
weekofyear             2876
dtype: int64

In [13]:
df1= df.interpolate(method='ffill')
df1

,MeasurementDateTime,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3,dayofweek,quarter,month,year,dayofyear,dayofmonth,datehour,weekofyear
date,,,,,,,,,,,,,,,,
2013-01-27 00:00:00,2013-01-27 00:00:00,114.0,1.7,18.1,5.0,100.0,0.017000,2.600000e-02,6.0,1.0,1.0,2013.0,27.0,27.0,0.0,4.0
2013-01-27 01:00:00,2013-01-27 01:00:00,151.0,0.4,17.8,3.0,89.0,0.086000,1.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,1.0,4.0
2013-01-27 02:00:00,2013-01-27 02:00:00,175.0,0.3,17.8,2.0,88.0,0.053000,2.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,2.0,4.0
2013-01-27 03:00:00,2013-01-27 03:00:00,264.0,0.8,16.8,2.0,92.0,0.072000,1.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,3.0,4.0
2013-01-27 04:00:00,2013-01-27 04:00:00,187.0,0.4,16.2,2.0,94.0,0.063000,1.000000e-03,6.0,1.0,1.0,2013.0,27.0,27.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,2015-12-31 18:00:00,85.0,1.4,14.6,10.0,76.0,0.076199,1.747050e-03,3.0,4.0,12.0,2015.0,365.0,31.0,18.0,53.0
2015-12-31 19:00:00,2015-12-31 19:00:00,99.0,1.3,14.6,9.0,75.0,0.098153,1.108100e-04,3.0,4.0,12.0,2015.0,365.0,31.0,19.0,53.0
2015-12-31 20:00:00,2015-12-31 20:00:00,210.0,0.7,14.5,10.0,79.0,0.106758,8.930000e-06,3.0,4.0,12.0,2015.0,365.0,31.0,20.0,53.0


In [14]:
cols =['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'datehour', 'weekofyear', 'WD-Hour', 'WS-Hour', 'Temp-Hour', 'SR-Hour', 'RH-Hour', 'NO2', 'O3']
df1 = df1[cols]
df1

,dayofweek,quarter,month,year,dayofyear,dayofmonth,datehour,weekofyear,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3
date,,,,,,,,,,,,,,,
2013-01-27 00:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,0.0,4.0,114.0,1.7,18.1,5.0,100.0,0.017000,2.600000e-02
2013-01-27 01:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,1.0,4.0,151.0,0.4,17.8,3.0,89.0,0.086000,1.000000e-03
2013-01-27 02:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,2.0,4.0,175.0,0.3,17.8,2.0,88.0,0.053000,2.000000e-03
2013-01-27 03:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,3.0,4.0,264.0,0.8,16.8,2.0,92.0,0.072000,1.000000e-03
2013-01-27 04:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,4.0,4.0,187.0,0.4,16.2,2.0,94.0,0.063000,1.000000e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,18.0,53.0,85.0,1.4,14.6,10.0,76.0,0.076199,1.747050e-03
2015-12-31 19:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,19.0,53.0,99.0,1.3,14.6,9.0,75.0,0.098153,1.108100e-04
2015-12-31 20:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,20.0,53.0,210.0,0.7,14.5,10.0,79.0,0.106758,8.930000e-06


In [15]:
df1.isna().sum()

dayofweek     0
quarter       0
month         0
year          0
dayofyear     0
dayofmonth    0
datehour      0
weekofyear    0
WD-Hour       0
WS-Hour       0
Temp-Hour     0
SR-Hour       0
RH-Hour       0
NO2           0
O3            0
dtype: int64

In [16]:
#cols =['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'datehour', 'weekofyear','MeasurementDateTime', 'WD-Hour', 'WS-Hour', 'Temp-Hour', 'SR-Hour', 'RH-Hour', 'NO2', 'O3']
#df = df[cols]
#df = df.set_index('MeasurementDateTime')
#df

In [17]:
#df1 = df1.drop(['year'],axis=1)
#df1

In [18]:
from xgboost import XGBRegressor

xgb = (XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=1,
               monotone_constraints='()', n_estimators=290,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=123,
              reg_alpha=0.05, reg_lambda=0.1))

In [19]:
df1

,dayofweek,quarter,month,year,dayofyear,dayofmonth,datehour,weekofyear,WD-Hour,WS-Hour,Temp-Hour,SR-Hour,RH-Hour,NO2,O3
date,,,,,,,,,,,,,,,
2013-01-27 00:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,0.0,4.0,114.0,1.7,18.1,5.0,100.0,0.017000,2.600000e-02
2013-01-27 01:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,1.0,4.0,151.0,0.4,17.8,3.0,89.0,0.086000,1.000000e-03
2013-01-27 02:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,2.0,4.0,175.0,0.3,17.8,2.0,88.0,0.053000,2.000000e-03
2013-01-27 03:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,3.0,4.0,264.0,0.8,16.8,2.0,92.0,0.072000,1.000000e-03
2013-01-27 04:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,4.0,4.0,187.0,0.4,16.2,2.0,94.0,0.063000,1.000000e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,18.0,53.0,85.0,1.4,14.6,10.0,76.0,0.076199,1.747050e-03
2015-12-31 19:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,19.0,53.0,99.0,1.3,14.6,9.0,75.0,0.098153,1.108100e-04
2015-12-31 20:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,20.0,53.0,210.0,0.7,14.5,10.0,79.0,0.106758,8.930000e-06


In [20]:
from sklearn.model_selection import train_test_split
df1.columns

Index(['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'datehour', 'weekofyear', 'WD-Hour', 'WS-Hour', 'Temp-Hour', 'SR-Hour', 'RH-Hour', 'NO2', 'O3'], dtype='object')

In [21]:
df1.columns = df1.columns.str.replace('WD-Hour', 'WD_Hour')
df1.columns = df1.columns.str.replace('WS-Hour', 'WS_Hour')
df1.columns = df1.columns.str.replace('Temp-Hour', 'Temp_Hour')
df1.columns = df1.columns.str.replace('SR-Hour', 'SR_Hour')
df1.columns = df1.columns.str.replace('RH-Hour', 'RH_Hour')
df1

,dayofweek,quarter,month,year,dayofyear,dayofmonth,datehour,weekofyear,WD_Hour,WS_Hour,Temp_Hour,SR_Hour,RH_Hour,NO2,O3
date,,,,,,,,,,,,,,,
2013-01-27 00:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,0.0,4.0,114.0,1.7,18.1,5.0,100.0,0.017000,2.600000e-02
2013-01-27 01:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,1.0,4.0,151.0,0.4,17.8,3.0,89.0,0.086000,1.000000e-03
2013-01-27 02:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,2.0,4.0,175.0,0.3,17.8,2.0,88.0,0.053000,2.000000e-03
2013-01-27 03:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,3.0,4.0,264.0,0.8,16.8,2.0,92.0,0.072000,1.000000e-03
2013-01-27 04:00:00,6.0,1.0,1.0,2013.0,27.0,27.0,4.0,4.0,187.0,0.4,16.2,2.0,94.0,0.063000,1.000000e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,18.0,53.0,85.0,1.4,14.6,10.0,76.0,0.076199,1.747050e-03
2015-12-31 19:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,19.0,53.0,99.0,1.3,14.6,9.0,75.0,0.098153,1.108100e-04
2015-12-31 20:00:00,3.0,4.0,12.0,2015.0,365.0,31.0,20.0,53.0,210.0,0.7,14.5,10.0,79.0,0.106758,8.930000e-06


In [22]:
cols = ['year','month','quarter','dayofyear','dayofmonth','weekofyear','dayofweek','datehour','WD_Hour', 'WS_Hour', 'Temp_Hour', 'SR_Hour', 'RH_Hour', 'NO2', 'O3']
df1 = df1[cols]
df1

,year,month,quarter,dayofyear,dayofmonth,weekofyear,dayofweek,datehour,WD_Hour,WS_Hour,Temp_Hour,SR_Hour,RH_Hour,NO2,O3
date,,,,,,,,,,,,,,,
2013-01-27 00:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,0.0,114.0,1.7,18.1,5.0,100.0,0.017000,2.600000e-02
2013-01-27 01:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,1.0,151.0,0.4,17.8,3.0,89.0,0.086000,1.000000e-03
2013-01-27 02:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,2.0,175.0,0.3,17.8,2.0,88.0,0.053000,2.000000e-03
2013-01-27 03:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,3.0,264.0,0.8,16.8,2.0,92.0,0.072000,1.000000e-03
2013-01-27 04:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,4.0,187.0,0.4,16.2,2.0,94.0,0.063000,1.000000e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,2015.0,12.0,4.0,365.0,31.0,53.0,3.0,18.0,85.0,1.4,14.6,10.0,76.0,0.076199,1.747050e-03
2015-12-31 19:00:00,2015.0,12.0,4.0,365.0,31.0,53.0,3.0,19.0,99.0,1.3,14.6,9.0,75.0,0.098153,1.108100e-04
2015-12-31 20:00:00,2015.0,12.0,4.0,365.0,31.0,53.0,3.0,20.0,210.0,0.7,14.5,10.0,79.0,0.106758,8.930000e-06


In [23]:
X, y = df1.iloc[:, :-1], df1.iloc[:, -1]

In [24]:
X

,year,month,quarter,dayofyear,dayofmonth,weekofyear,dayofweek,datehour,WD_Hour,WS_Hour,Temp_Hour,SR_Hour,RH_Hour,NO2
date,,,,,,,,,,,,,,
2013-01-27 00:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,0.0,114.0,1.7,18.1,5.0,100.0,0.017000
2013-01-27 01:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,1.0,151.0,0.4,17.8,3.0,89.0,0.086000
2013-01-27 02:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,2.0,175.0,0.3,17.8,2.0,88.0,0.053000
2013-01-27 03:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,3.0,264.0,0.8,16.8,2.0,92.0,0.072000
2013-01-27 04:00:00,2013.0,1.0,1.0,27.0,27.0,4.0,6.0,4.0,187.0,0.4,16.2,2.0,94.0,0.063000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 18:00:00,2015.0,12.0,4.0,365.0,31.0,53.0,3.0,18.0,85.0,1.4,14.6,10.0,76.0,0.076199
2015-12-31 19:00:00,2015.0,12.0,4.0,365.0,31.0,53.0,3.0,19.0,99.0,1.3,14.6,9.0,75.0,0.098153
2015-12-31 20:00:00,2015.0,12.0,4.0,365.0,31.0,53.0,3.0,20.0,210.0,0.7,14.5,10.0,79.0,0.106758


In [25]:
y

date
2013-01-27 00:00:00    2.600000e-02
2013-01-27 01:00:00    1.000000e-03
2013-01-27 02:00:00    2.000000e-03
2013-01-27 03:00:00    1.000000e-03
2013-01-27 04:00:00    1.000000e-03
                           ...     
2015-12-31 18:00:00    1.747050e-03
2015-12-31 19:00:00    1.108100e-04
2015-12-31 20:00:00    8.930000e-06
2015-12-31 21:00:00    1.000000e-08
2015-12-31 22:00:00    0.000000e+00
Freq: H, Name: O3, Length: 25655, dtype: float64

In [26]:
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.2, random_state = 123)

In [27]:
xgb.fit(train_X, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=290,
             n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=123,
             reg_alpha=0.05, reg_lambda=0.1, ...)

In [28]:
pred = xgb.predict(test_X)
pred

array([0.00045016, 0.02260762, 0.0020332 , ..., 0.03950962, 0.06227764,
       0.01274938], dtype=float32)

In [29]:
# RMSE Computation
from sklearn.metrics import mean_squared_error as MSE
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  0.005350


In [30]:
xgb.score(train_X, train_y)

0.9756177779346994

In [31]:
xgb.score(test_X, test_y)


0.918161859103899

In [32]:
df1.columns

Index(['year', 'month', 'quarter', 'dayofyear', 'dayofmonth', 'weekofyear', 'dayofweek', 'datehour', 'WD_Hour', 'WS_Hour', 'Temp_Hour', 'SR_Hour', 'RH_Hour', 'NO2', 'O3'], dtype='object')

In [33]:
xgb.predict(test_X)

array([0.00045016, 0.02260762, 0.0020332 , ..., 0.03950962, 0.06227764,
       0.01274938], dtype=float32)

In [34]:
test = [2015,8,3,214,2,31,6,15,118,2.8,48.4,718,18,0.031]
p = pd.DataFrame([test])
p


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2015,8,3,214,2,31,6,15,118,2.8,48.4,718,18,0.031


In [35]:
xgb.predict(p)

array([0.14080931], dtype=float32)

In [36]:
#import pickle
#file_name = "xgb_reg_sklearn_updated.pkl"

# save
#pickle.dump(xgb, open(file_name, "wb"))

In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25655 entries, 2013-01-27 00:00:00 to 2015-12-31 22:00:00
Freq: H
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        25655 non-null  float64
 1   month       25655 non-null  float64
 2   quarter     25655 non-null  float64
 3   dayofyear   25655 non-null  float64
 4   dayofmonth  25655 non-null  float64
 5   weekofyear  25655 non-null  float64
 6   dayofweek   25655 non-null  float64
 7   datehour    25655 non-null  float64
 8   WD_Hour     25655 non-null  float64
 9   WS_Hour     25655 non-null  float64
 10  Temp_Hour   25655 non-null  float64
 11  SR_Hour     25655 non-null  float64
 12  RH_Hour     25655 non-null  float64
 13  NO2         25655 non-null  float64
 14  O3          25655 non-null  float64
dtypes: float64(15)
memory usage: 3.1 MB


In [38]:
#evaluate_model(tuned_blender)

In [39]:
#evaluate_model(tuned_lightgbm)

In [40]:
#df

In [41]:
#df.index.names = ['date']
#df

In [42]:
#df.isna().sum()

In [43]:
#df =df.reset_index()
#df.isna().sum()

In [44]:
#df = df.dropna(how='any')
#df

In [45]:
#df = df.set_index('date')
#df

In [46]:
#df

In [47]:
#df = df.reset_index()
#df

In [48]:
#df = df.set_index('date').asfreq('h')
#df

In [49]:
#df.isna().sum()

In [50]:
#df1= df.interpolate()
#df1

In [51]:
#df1.isna().sum()

In [52]:
#df1

In [53]:
#df5=df1[df1.index.year == 2015]
#df5

In [54]:
#save_model(final_lgbm,'Final_Lgbm_Model_CC_no_num_prod')